# Import Module

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

import numpy as np

import models
import data
from si4dnn import cam_si

2022-09-29 18:50:44.109870: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/miwa.daiki/.local/lib:/home/miwa.daiki/.local/lib::/usr/local/cuda/extras/CUPTI/lib64:/home/miwa.daiki/.local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/home/miwa.daiki/.local/cuda/lib64
2022-09-29 18:50:44.109898: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Define and Train Model

In [18]:
d = 16
shape = (d,d,1)

# define simple CNN for classification 
inputs = keras.layers.Input(shape=shape)
conv1 = keras.layers.Conv2D(4,(3,3),padding="same", activation="relu")(inputs)
conv2 = keras.layers.Conv2D(4,(3,3),padding="same",activation="relu")(conv1)
maxpool1 = keras.layers.MaxPool2D((2,2))(conv2)
conv3 = keras.layers.Conv2D(4,(3,3),padding="same", activation="relu")(maxpool1)
conv4 = keras.layers.Conv2D(4,(3,3),padding="same",activation="relu")(conv3)
up1 = keras.layers.UpSampling2D((2,2))(conv4)
gap1 = keras.layers.GlobalAveragePooling2D()(up1)
dense1 = keras.layers.Dense(1,activation="sigmoid")(gap1)
model = keras.Model(inputs=inputs,outputs=dense1)

# Train model
X,Y = data.generate_data_classification(shape,1000,0,2)
model.compile(optimizer="adam",loss="binary_crossentropy", metrics=["accuracy"])
checkpointer = tf.keras.callbacks.ModelCheckpoint(
f"./classification_model/model_{d}" + ".h5", verbose=1, save_best_only=True
)
results = model.fit(
    X, Y, validation_split=0.3, epochs=100, callbacks=[checkpointer]
)

# Add Cam Component
layers = model.layers
cam = models.CAM(
    layers[-1], shape)([layers[-3].output, layers[-1].output])
model_with_cam = tf.keras.Model(inputs=model.input, outputs=cam)

# Selective Inference for Saliency Map 

In [35]:
# generate data
X = data.generate_data_classification(shape,1,0,1,True)
X_ref = data.generate_data_classification(shape,1,0,1,True)
output = model_with_cam.predict(X)[0]

In [ ]:
# compute selective p value for Mean Null Test
# Once you defined model which output CAM with Keras, you can compute p value by following code

cam_si_thr = cam_si.si4dnn_cam_si_thr_abs(model_with_cam,thr=thr)
p_value  = cam_si_thr.inference(X,ref=X_ref)[0]